<a href="https://colab.research.google.com/github/pwjdgus/Data_Analytics_for_Age_friendly_busan/blob/bae4/%EA%B2%BD%EC%82%AC%EB%8F%84%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install googlemaps
!pip install haversine
import requests
import googlemaps
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for googlemaps: filename=googlemaps-4.6.0-py3-none-any.whl size=38554 sha256=dc67cfe82932d29b7878c8f0a1a8198aa75df4686e9c74c94a0a2225a526d36e
  Stored in directory: /root/.cache/pip/wheels/80/db/c0/6d958585fa97b20e250bf437acf7e6e715b4809c2dd4e55367
Successfully built googlemaps
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# 중심지 [위도, 경도, 고도]
kizang_pos= [35.23876594878238, 129.21575390899562, 28.18951225280762]
Jeonggwan_pos = [35.3231, 129.1784, 59.41537094116211]
Noksan_pos= [35.08562495305441, 128.87931646088754, 0]
Myungji_pos = [35.10820208025592, 128.9264932285998, 0]

In [ ]:
# google 지오코딩 API를 이용해서 위도, 경도 데이터 가져오기

# my_key = "본인이 발급받은 API키를 입력"
my_key = "(본인의 API키 입력)"

# 구글맵스 객체 생성하기
maps = googlemaps.Client(key=my_key)  # my_key값 입력
lat = []  # 위도
lng = []  # 경도

# 장소(또는 주소) 리스트
places = ["부산광역시 기장군 기장읍 차성로 288번길 9-1"]

i = 0
for place in places:
    i = i + 1
    try:
        print(i, place)

    # 지오코딩 API 결과값 호출하여 geo_location 변수에 저장
        geo_location = maps.geocode(place)[0].get('geometry')
        lat.append(geo_location['location']['lat'])
        lng.append(geo_location['location']['lng'])

    except:
        lat.append('')
        lng.append('')
        print(i)

# 데이터프레임으로 변환하기
df = pd.DataFrame({'위도': lat, '경도': lng}, index=places)
print(df)

1 부산광역시 기장군 기장읍 차성로 288번길 9-1
                                    위도          경도
부산광역시 기장군 기장읍 차성로 288번길 9-1  35.243387  129.214164


In [ ]:
import json
import math
from haversine import haversine

In [ ]:
# 구글 api 이용하여 고도 계산
url = f"https://maps.googleapis.com/maps/api/elevation/json?locations={lat[0]}%2C{lng[0]}&key=(본인의 API키 입력)"
    
response = requests.get(url)
jsonObj = json.loads(response.text)

elevation = jsonObj["results"][0]["elevation"] # 고도 값 url로부터 불러오기

print("고도 :",elevation)

고도 : 17.81886863708496


In [ ]:
#코랩에서 csv파일 읽어서 고도 구하는 식
import csv, json
import requests
import pandas as pd

# 고도 구하는 메서드
# colab에 csv을 업로드 후 사용
def findElevation(data) :           
    latitude = float(data[4])       # 위도가 적혀있는 열을 index로 설정
    longitude = float(data[5])      # 경도가 적혀있는 열을 index로 설정
    url = f"https://maps.googleapis.com/maps/api/elevation/json?locations={latitude}%2C{longitude}4&key=(본인의 API키 입력)" 
    response = requests.get(url)
    jsonObj = json.loads(response.text)
    elevation = jsonObj["results"][0]["elevation"]
    return elevation                

f = open('/content/부산광역시_경로당 현황_20201231.csv', encoding='cp949')  
reader = csv.reader(f)              #csv파일 읽어오기

data = list()                       # 고도 저장하는 list
for i in reader :
  data.append(findElevation(i))
print(data)

[16.98797798156738, 24.73167991638184, 31.67792701721191, -0.8516998887062073, 7.791562557220459, 11.08033752441406, 8.767248153686523, 0, 0, 1.966750860214233, 6.455269336700439, 0, 10.40573215484619, 0, 0, 0, 0.005504369735717773, 0, 5.86199951171875, 0]


In [ ]:
# haversine 라이브러리 이용하여 두 지점(위도/경도) 간 직선 거리 구하기
# 중심지(기장군) - 구한 장소(부산광역시 기장군 기장읍 차성로 288번길 9-1) 간 직선 거리
straight_way = haversine((kizang_pos[0],kizang_pos[1]), (float(lat[0]),float(lng[0])), unit='m')

# 고도 차
h = math.sqrt((kizang_pos[2] - elevation)**2) # 음수 값 나오지 않게 하기 위해 제곱 후 루트

# 경사 길이
slope = math.sqrt(straight_way**2 + h**2) # 피타고라스 이용

# 경사도
rad = math.atan2(h, straight_way) # 아크 탄젠트(역함수)를 이용하여 각도를 라디안 값으로 구함.
deg = (rad*180)/math.pi # 라디안 값을 degree로 변형

print("경사도 :",deg)

경사도 : 1.1130362636742126


In [ ]:
print("hi")